# **1.Introdução ao Spark e DataFrames**

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, upper, avg

## Criar uma SparkSession

In [3]:
spark = SparkSession.builder.appName("ExercicioSpark").getOrCreate()

## Ler o arquivo CSV e criar um DataFrame
### Certifique-se de ajustar o caminho do arquivo para o local correto

In [4]:
df = spark.read.csv("dados.csv", header=True, inferSchema=True)

## Exibir as primeiras 5 linhas do DataFrame

In [5]:
print("Primeiras 5 linhas do DataFrame:")
df.show(5)

Primeiras 5 linhas do DataFrame:
+--------------+-----+------------+
|          nome|idade|      cidade|
+--------------+-----+------------+
|     Ana Souza|   28|    Curitiba|
|   Carlos Lima|   35|    Salvador|
| Mariana Alves|   22|   Fortaleza|
|    João Pedro|   41|Porto Alegre|
|Fernanda Rocha|   30|      Recife|
+--------------+-----+------------+
only showing top 5 rows



## Filtrar as pessoas com idade acima de 25

In [6]:
print("Pessoas com idade acima de 25:")
df.filter(col("idade") > 25).show()

Pessoas com idade acima de 25:
+---------------+-----+--------------+
|           nome|idade|        cidade|
+---------------+-----+--------------+
|      Ana Souza|   28|      Curitiba|
|    Carlos Lima|   35|      Salvador|
|     João Pedro|   41|  Porto Alegre|
| Fernanda Rocha|   30|        Recife|
|    Bruno Costa|   27|Belo Horizonte|
|Luciana Martins|   33|Rio de Janeiro|
| Diego Ferreira|   45|      Campinas|
| Isabela Mendes|   26|         Natal|
|Rafael Oliveira|   38|     São Paulo|
+---------------+-----+--------------+



# **2.Transformações e Ações**

## Selecionar apenas as colunas "nome" e "cidade"

In [7]:
df_selecionado = df.select("nome", "cidade")
print("Colunas selecionadas:")
df_selecionado.show()

Colunas selecionadas:
+---------------+--------------+
|           nome|        cidade|
+---------------+--------------+
|      Ana Souza|      Curitiba|
|    Carlos Lima|      Salvador|
|  Mariana Alves|     Fortaleza|
|     João Pedro|  Porto Alegre|
| Fernanda Rocha|        Recife|
|    Bruno Costa|Belo Horizonte|
|Luciana Martins|Rio de Janeiro|
| Diego Ferreira|      Campinas|
| Isabela Mendes|         Natal|
|Rafael Oliveira|     São Paulo|
+---------------+--------------+



## Ordenar os dados pela coluna "idade" em ordem decrescente

In [8]:
df_ordenado = df.orderBy(col("idade").desc())
print("Dados ordenados pela idade (decrescente):")
df_ordenado.show()

Dados ordenados pela idade (decrescente):
+---------------+-----+--------------+
|           nome|idade|        cidade|
+---------------+-----+--------------+
| Diego Ferreira|   45|      Campinas|
|     João Pedro|   41|  Porto Alegre|
|Rafael Oliveira|   38|     São Paulo|
|    Carlos Lima|   35|      Salvador|
|Luciana Martins|   33|Rio de Janeiro|
| Fernanda Rocha|   30|        Recife|
|      Ana Souza|   28|      Curitiba|
|    Bruno Costa|   27|Belo Horizonte|
| Isabela Mendes|   26|         Natal|
|  Mariana Alves|   22|     Fortaleza|
+---------------+-----+--------------+



## Converter os nomes para letras maiúsculas

In [9]:
df_maiusculo = df.withColumn("nome", upper(col("nome")))
print("Nomes em letras maiúsculas:")
df_maiusculo.show()


Nomes em letras maiúsculas:
+---------------+-----+--------------+
|           nome|idade|        cidade|
+---------------+-----+--------------+
|      ANA SOUZA|   28|      Curitiba|
|    CARLOS LIMA|   35|      Salvador|
|  MARIANA ALVES|   22|     Fortaleza|
|     JOÃO PEDRO|   41|  Porto Alegre|
| FERNANDA ROCHA|   30|        Recife|
|    BRUNO COSTA|   27|Belo Horizonte|
|LUCIANA MARTINS|   33|Rio de Janeiro|
| DIEGO FERREIRA|   45|      Campinas|
| ISABELA MENDES|   26|         Natal|
|RAFAEL OLIVEIRA|   38|     São Paulo|
+---------------+-----+--------------+



# **3. Agregações Simples**

## Calcular a idade média das pessoas no DataFrame usando .agg() e avg

In [10]:
idade_media = df.agg(avg("idade").alias("idade_media"))
print("Idade média das pessoas:")
idade_media.show()

Idade média das pessoas:
+-----------+
|idade_media|
+-----------+
|       32.5|
+-----------+



## Contar o número de pessoas por cidade usando .groupBy() e .count()

In [11]:
pessoas_por_cidade = df.groupBy("cidade").count().alias("numero_pessoas")
print("Número de pessoas por cidade:")
pessoas_por_cidade.show()

Número de pessoas por cidade:
+--------------+-----+
|        cidade|count|
+--------------+-----+
|     Fortaleza|    1|
|      Curitiba|    1|
|      Campinas|    1|
|Belo Horizonte|    1|
|      Salvador|    1|
|     São Paulo|    1|
|         Natal|    1|
|        Recife|    1|
|Rio de Janeiro|    1|
|  Porto Alegre|    1|
+--------------+-----+



# **4. Spark SQL**

## Registrar o DataFrame como uma tabela temporária

In [12]:
df.createOrReplaceTempView("tabela_pessoas")

## Escrever uma consulta SQL para selecionar todas as pessoas que moram em uma cidade específica

In [13]:
cidade_especifica = "São Paulo" 

consulta_cidade = f""" 
SELECT *
FROM tabela_pessoas
WHERE cidade = '{cidade_especifica}'
""" 

pessoas_na_cidade = spark.sql(consulta_cidade) 

print(f"Pessoas que moram na cidade '{cidade_especifica}':") 
pessoas_na_cidade.show()

Pessoas que moram na cidade 'São Paulo':
+---------------+-----+---------+
|           nome|idade|   cidade|
+---------------+-----+---------+
|Rafael Oliveira|   38|São Paulo|
+---------------+-----+---------+



## Escrever uma consulta SQL para calcular a soma das idades

In [14]:
consulta_soma_idades = """
SELECT SUM(idade)
AS soma_das_idades
FROM tabela_pessoas 
""" 
soma_idades = spark.sql(consulta_soma_idades) 

print("Soma das idades:") 
soma_idades.show()

Soma das idades:
+---------------+
|soma_das_idades|
+---------------+
|            325|
+---------------+



# **5.Salvar DataFrame**

In [15]:
caminho_parquet = "data"
df.write.mode("overwrite").parquet(caminho_parquet)
print(f"✅ DataFrame salvo como Parquet em: {caminho_parquet}")

Py4JJavaError: An error occurred while calling o69.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja